In [ ]:
%pip install -r requirements.txt

In [ ]:
%pip install "unsloth[cu121-ampere] @ git+https://github.com/unslothai/unsloth.git"

In [3]:
from lora import Lora_FineTuner

In [4]:
bucket='llm-storage-bucket'
data_key = 'Cosmetics_dataset_0.8/'
dataset_path = 's3://{}/{}'.format(bucket, data_key)

In [5]:
llm_trainer = Lora_FineTuner(dataset=dataset_path,
                             model_name="unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
                             max_seq_length=140
                             )

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/fsspec/registry.py:272: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify a lower version bound on s3fs, or
update the current installation.

  warnings.warn(s3_msg)


config.json:   0%|          | 0.00/1.07k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Mistral patching release 2024.3
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.1.0. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.22.post7. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors:   0%|          | 0.00/4.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

Unsloth 2024.3 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [6]:
llm_trainer.train(n_epochs=3, 
                  device_batch_size=16, 
                  save_steps= 50,
                  resume = True,
                  # n_rows=1000,
                 )

Map (num_proc=2):   0%|          | 0/9436 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/2442 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 9,436 | Num Epochs = 3
O^O/ \_/ \    Batch size per device = 16 | Gradient Accumulation steps = 4
\        /    Total batch size = 64 | Total steps = 441
 "-____-"     Number of trainable parameters = 41,943,040
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········


wandb: Appending key for api.wandb.ai to your netrc file: /home/ec2-user/.netrc


Step,Training Loss


In [6]:
1+1

2

### Saving for VLLM

In [7]:
import datetime
now = datetime.datetime.now()

In [8]:
model_save_path_16bit = f"Cosmetics_Expert_LLM_16bit_{now.strftime('%Y-%m-%d_%H%M%S')}"

In [9]:
llm_trainer.model.save_pretrained_merged(model_save_path_16bit, llm_trainer.tokenizer, save_method = "merged_16bit",)

Unsloth: You have 2 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which will take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 9.49 out of 15.32 RAM for saving.


100%|██████████| 32/32 [02:27<00:00,  4.60s/it]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Unsloth: Saving Cosmetics_Expert_LLM_16bit_2024-03-22_143522/pytorch_model-00001-of-00003.bin...
Unsloth: Saving Cosmetics_Expert_LLM_16bit_2024-03-22_143522/pytorch_model-00002-of-00003.bin...
Unsloth: Saving Cosmetics_Expert_LLM_16bit_2024-03-22_143522/pytorch_model-00003-of-00003.bin...
Done.


### Upload to S3

In [15]:
import boto3
import os

In [16]:
session = boto3.Session()

In [17]:
models_key = f"Models/{model_save_path_16bit}/"
model_to_upload = f"./{model_save_path_16bit}/"

In [18]:
for file in os.listdir(model_to_upload):
    file_path = os.path.join(model_to_upload, file)
    if os.path.isfile(file_path):
        # Upload the file to S3
        session.resource('s3').Bucket(bucket).Object(os.path.join(models_key, file)).upload_file(file_path)


KeyboardInterrupt: 

In [ ]:
1+1